# gan

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten

In [22]:
# 画像(0~9の数値)認識のためのデータセット
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train_full, y_train_full), (x_test, y_test) = fashion_mnist.load_data()

# クラスの種類
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [23]:
# 検証データとテストデータに分割及び、画像データは 0-1 の範囲に修正
x_valid, x_train = x_train_full[:5000] / 255, x_train_full[5000:] / 255
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# データサイズの確認
print(x_train.shape, x_test.shape, x_valid.shape)
print(y_train.shape, y_test.shape, y_valid.shape)

(55000, 28, 28) (10000, 28, 28) (5000, 28, 28)
(55000,) (10000,) (5000,)


In [21]:
cording_size = 30

generator = Sequential([
    Input(shape=[cording_size]),
    Dense(100, activation='selu'),
    Dense(150, activation='selu'),
    Dense(28*28, activation='sigmoid'),
    Reshape([28, 28]),
])

discriminator = Sequential([
    Input(shape=[28, 28]),
    Flatten(),
    Dense(150, activation='selu'),
    Dense(100, activation='selu'),
    Dense(1, activation='sigmoid'),
])

gen = Sequential([generator, discriminator])

discriminator.compile(loss='binary_crossentropy', optimizer='rmsprop')
discriminator.trainable = False
gen.compile(loss='binary_crossentropy', optimizer='rmsprop')

In [26]:
batch_size = 12
dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(1000)
dataset = dataset.batch(batch_size=batch_size, drop_remainder=True).prefetch(1)

In [33]:
def train_gen(gen, dataset, batch_size, cording_size, n_epochs=50):
    generator, discriminator = gen.layers
    for epoch in range(n_epochs):
        print(f"Epoch {epoch+1}/{n_epochs}")
        for x_batch in dataset:
            x_batch = tf.cast(x_batch, tf.float32)

            # 1st Step
            noise = tf.random.normal(shape=[batch_size, cording_size])
            generated_images = generator(noise)
            x_fake_and_real = tf.concat([generated_images, x_batch], axis=0)
            y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
            discriminator.trainable = True
            discriminator.train_on_batch(x_fake_and_real, y1)

            # 2nd Step
            noise = tf.random.normal(shape=[batch_size, cording_size])
            y2 = tf.constant([[1.]] * batch_size)
            discriminator.trainable = False
            gen.train_on_batch(noise, y2)

train_gen(gen, dataset, batch_size, cording_size)



Epoch 1/50


KeyboardInterrupt: 